# Lab01 using openAI library

https://developers.googleblog.com/en/gemini-is-now-accessible-from-the-openai-library/

In [143]:
# Libraries version
# genai.__version__ == 0.8.3
# openai.__version__ == 1.54.4
# langchain.__version__ == 0.3.7

In [1]:
from dotenv import load_dotenv
import os
import google.generativeai as genai  
from openai import OpenAI
from IPython.display import HTML, Markdown, display

# Load the variables from the .env file
load_dotenv()

# Access the variables
gemini_api_key = os.getenv("GOOGLE_API_KEY")


client = OpenAI(
    api_key=gemini_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/"
)


def ask_model(msgs, llm, **kwargs):
    response = client.chat.completions.create(
    model=llm,
    n=1,
    messages=msgs,
    **kwargs
    )
    return response.choices[0].message



# 1. Example: api call

In [2]:
flash = "gemini-1.5-flash"

user_prompt = "Explain AI to me like I'm a kid."
messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_prompt}
            ]

answer = ask_model(messages, flash)

Markdown(answer.content)

Imagine you have a really smart friend who loves to learn new things. That friend is like AI, or Artificial Intelligence. 

AI is like a computer program that can learn and solve problems just like you can! It can learn from lots of information, like reading books or watching videos, and then use that information to answer questions, play games, or even write stories!

Think about how you learn to ride a bike: you practice and get better over time. AI is the same way. It keeps practicing and gets better at what it does, like recognizing your face in a photo or playing your favorite video game.

AI is already helping us in lots of ways. It can translate languages, tell you what the weather will be like, and even help doctors diagnose illnesses. 

But AI is still learning and growing, just like you!  It's exciting to see what amazing things AI will be able to do in the future! 


# 2. Multi-Turn chat and history tracking

In [3]:
# Initialize conversation history
conversation_history = [
    {"role": "system", "content": "You are a helpful assistant."}
]


def chat_with_openai(prompt, history, model, **kwargs):
    # Add user's message to the history
    history.append({"role": "user", "content": prompt})
    
    response = ask_model(history, model, **kwargs)
    
    # Get the assistant's response
    assistant_message = response.content
    
    # Add assistant's message to the history
    history.append({"role": "assistant", "content": assistant_message})
    
    return assistant_message, history


# First user prompt
user_prompt = 'Hello! My name is Zlork.'
response, conversation_history = chat_with_openai(user_prompt, conversation_history, flash)
print(f"User Prompt: {user_prompt}")
print(f"Assistant: \n{response}")
print("--")
# Second user prompt
user_prompt = "Can you tell something interesting about dinosaurs?"
response, conversation_history = chat_with_openai(user_prompt, conversation_history, flash)
print(f"User Prompt: {user_prompt}")
print(f"Assistant: \n{response}")
print("--")
# Third user prompt
user_prompt = "Do you remember what my name is?"
response, conversation_history = chat_with_openai(user_prompt, conversation_history, flash)
print(f"User Prompt: {user_prompt}")
print(f"Assistant: \n{response}")

User Prompt: Hello! My name is Zlork.
Assistant: 
Hello Zlork! It's nice to meet you. 😊 What can I do for you today? 

--
User Prompt: Can you tell something interesting about dinosaurs?
Assistant: 
You're in for a treat, Zlork! Dinosaurs are full of fascinating facts.  Here's one: 

Did you know that some dinosaurs had feathers?  It's true! We used to think they were all scaly, but fossils have shown that many species, like Velociraptor and T-Rex, had feathers, although some were more like downy fuzz than the flight feathers of birds today. 

This discovery tells us that dinosaurs were likely more bird-like than we thought, and it gives us a glimpse into their evolution. What else would you like to know about dinosaurs? 

--
User Prompt: Do you remember what my name is?
Assistant: 
Of course I remember! You're Zlork.  😊 I'm a pretty good listener, especially when it comes to names.  Is there anything else I can help you with, Zlork? 



# 3. Choose a Model

In [6]:
for model in genai.list_models():
  print(model.name)

KeyboardInterrupt: 

# 4. Model details

In [142]:
for model in genai.list_models():
  if model.name == 'models/gemini-1.5-flash':
    print(model)
    break

KeyboardInterrupt: 

# 5. Explore generation parameters

## 5.1 number of tokens

In [8]:
model = "gemini-1.5-flash"

conversation_history = [
    {"role": "system", "content": "You are a helpful assistant."},
]
user_prompt = "Write a 1000 word essay on the importance of olives in modern society."

response, _ = chat_with_openai(user_prompt, conversation_history, model, max_tokens=200)

print(response)

## Beyond the Brine: The Enduring Importance of Olives in Modern Society

The olive, a small, unassuming fruit, holds a surprising amount of significance in modern society. Beyond its simple appearance lies a history rich in culture, tradition, and economic value. From the Mediterranean shores where it originated to the globalized markets where it is now enjoyed, the olive has woven itself into the fabric of human civilization, playing a critical role in nutrition, economy, and even environmental sustainability.

Perhaps the most recognized aspect of the olive's importance is its role in the human diet. For millennia, olives have been a staple food in the Mediterranean region, contributing to the famed "Mediterranean diet" that is now lauded for its health benefits. Olives are rich in monounsaturated fats, particularly oleic acid, which has been linked to heart health and a reduced risk of chronic diseases. They also contain valuable antioxidants, vitamins, and minerals, contributing t

## 5.2 Temperature

OpenAI's Python API client does not include a built-in retry mechanism like google.api_core. 
However, you can implement a retry mechanism yourself using Python's tenacity library, 
which is commonly used for handling retries with customizable policies.
!pip install tenacity

- *retry_if_exception_type*: Retries only if specific exceptions occur, such as RateLimitError, APIConnectionError, or Timeout.

- *stop_after_attempt(5)*: Stops retrying after 5 attempts.

- *wait_exponential*: Implements exponential backoff with a base of 4 seconds, doubling each retry, up to a maximum of 60 seconds.

In [9]:
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type
from openai import RateLimitError, APIConnectionError, Timeout, OpenAIError

# Define a retry strategy
@retry(
    retry=retry_if_exception_type((RateLimitError, APIConnectionError, Timeout)),
    stop=stop_after_attempt(5),  # Retry up to 5 times
    wait=wait_exponential(multiplier=1, min=4, max=60),  # Exponential backoff
)
def chat_with_openai_with_retry(prompt, history, model, **kwargs):
    response, history = chat_with_openai(prompt, history, model, **kwargs)
    try:
        response, history = chat_with_openai(prompt, history, model, **kwargs)
        return response, history
    except OpenAIError as e:
        print(f"An error occurred: {e}")
        raise  # Re-raise the error to trigger the retry

In [10]:
# higher Temperature

model = "gemini-1.5-flash"
temperature = 2
trial = 0

while trial <= 5:
    try:
        user_prompt = "Pick a random colour... (respond in a single word)"
        conversation_history = []
        response, _ = chat_with_openai_with_retry(user_prompt, conversation_history, model, temperature=temperature)
        print(response, '-' * 25)
        trial += 1
    except Exception as e:
        print(f"Final exception after retries: {e}")

Purple 
 -------------------------
Teal 
 -------------------------
Azure 
 -------------------------
Teal. 
 -------------------------
Purple 
 -------------------------
Teal 
 -------------------------


In [11]:
# Low temp

model = "gemini-1.5-flash"
temperature = 0

trial = 0

while trial <= 5:
    try:
        user_prompt = "Pick a random colour... (respond in a single word)"
        conversation_history = []
        response, _ = chat_with_openai_with_retry(user_prompt, conversation_history, model, temperature=temperature)
        print(response, '-' * 25)
        trial += 1
    except Exception as e:
        print(f"Final exception after retries: {e}")

Teal 
 -------------------------
Teal 
 -------------------------
Teal 
 -------------------------
Teal 
 -------------------------
Teal 
 -------------------------
Teal 
 -------------------------


## 5.3 Top-K and top-P

Like temperature, top-K and top-P parameters are also used to control the diversity of the model's output.

Top-K is a positive integer that defines the number of most probable tokens from which to select the output token. A top-K of 1 selects a single token, performing greedy decoding.

Top-P defines the probability threshold that, once cumulatively exceeded, tokens stop being selected as candidates. A top-P of 0 is typically equivalent to greedy decoding, and a top-P of 1 typically selects every token in the model's vocabulary.

When both are supplied, the Gemini API will filter top-K tokens first, then top-P and then finally sample from the candidate tokens using the supplied temperature.

Run this example a number of times, change the settings and observe the change in output.

In [12]:
# openai library does not support top_k directly

user_prompt = "Write a short story about a cat who goes on an adventure."
conversation_history = [
                        {"role": "system", "content": "You are a creative writer."}
        ]
params = dict(top_p=0.95, temperature=1.0)
response, _ = chat_with_openai_with_retry(user_prompt, 
                                          conversation_history, 
                                          model, 
                                          **params)

print(response)

Humphrey was a cat of routine. He woke at 6:30 am, demanded breakfast, napped in the sunbeam on the living room rug, and then proceeded to chase imaginary foes in the garden until dinner. This was his life, his world, his joy – a life of predictable, fluffy bliss. 

But one day, the wind changed. A gust, strong and sudden, ripped open the back door, sending Humphrey tumbling onto the patio. He landed with a surprised meow, his whiskers twitching, and found himself staring at… a box. Not just any box, mind you, a cardboard box, overflowing with colorful scarves. 

This was not in Humphrey’s routine. Curiosity, that insatiable feline itch, began to prickle. He sniffed the box cautiously, then poked his head inside. The scent of adventure, a heady mix of silk and faraway places, washed over him. He jumped in, his paws landing on a riot of colors.

The scarves, he discovered, were his ticket to another world. He squeezed between silk flowers, his whiskers brushing against smooth satin. He 

# 6. Prompting

# 6.1 Zero-shot prompting

In [13]:
# We remove the system role entirely, because the instruction is included in the user's prompt, 
# and the model only needs to respond to the classification.

conversation_history = []
params = dict(top_p=1, temperature=0.1, max_tokens=5)

zero_shot_prompt = """Classify movie reviews as POSITIVE, NEUTRAL or NEGATIVE.
Review: "Her" is a disturbing study revealing the direction
humanity is headed if AI is allowed to keep evolving,
unchecked. I wish there were more movies like this masterpiece.
Sentiment: """

response, _ = chat_with_openai_with_retry(zero_shot_prompt, 
                                          conversation_history, 
                                          model, 
                                          **params)

print(response)


Sentiment: **POSITIVE**


## 6.2 One shot and few shot

In [14]:
from textwrap import dedent


params = dict(top_p=1, temperature=0.1, max_tokens=250)

few_shot_prompt = """Parse a customer's pizza order into valid JSON:
EXAMPLE:
I want a small pizza with cheese, tomato sauce, and pepperoni.
JSON Response:
```
{
"size": "small",
"type": "normal",
"ingredients": ["cheese", "tomato sauce", "peperoni"]
}
```

EXAMPLE:
Can I get a large pizza with tomato sauce, basil and mozzarella
JSON Response:
```
{
"size": "large",
"type": "normal",
"ingredients": ["tomato sauce", "basil", "mozzarella"]
}

ORDER:
"""

conversation_history = [{"role": "system", "content": dedent(few_shot_prompt)}]
customer_order = "Give me a large with cheese & pineapple"

response, _ = chat_with_openai_with_retry(customer_order, 
                                          conversation_history, 
                                          model, 
                                          **params)

print(response)


```json
{
"size": "large",
"type": "normal",
"ingredients": ["cheese", "pineapple"]
}
``` 



## JSON mode

For simplicity we will use langchain

In [23]:
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function
from pydantic import BaseModel, Field


pizza_order_tool 

{'name': 'PizzaOrder',
 'description': 'A function to parse pizza order',
 'parameters': {'properties': {'size': {'description': 'The size of the pizza',
    'type': 'string'},
   'ingredients': {'description': 'List of ingredients on the pizza',
    'items': {'type': 'string'},
    'type': 'array'},
   'type': {'description': 'The type of pizza', 'type': 'string'}},
  'required': ['size', 'ingredients', 'type'],
  'type': 'object'}}

In [56]:
from langchain_core.utils.function_calling import convert_pydantic_to_openai_function
from pydantic import BaseModel, Field
import json


class PizzaOrder(BaseModel):
    """A function to parse pizza order"""
    size: str = Field(description="The size of the pizza")
    ingredients: list[str] = Field(description="List of ingredients on the pizza")
    type: str = Field(description="The type of pizza")


pizza_order_tool = convert_pydantic_to_openai_function(PizzaOrder)
response = client.chat.completions.create(
    model=model,
    messages=[{"role": "user", "content": "Can I have a large dessert pizza with apple and chocolate"}],
    tools=[{"type": "function", "function": pizza_order_tool}]
)

print("raw response ", response) 

raw response  ChatCompletion(id=None, choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='model', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='0', function=Function(arguments='{"type":"dessert","size":"large","ingredients":["apple","chocolate"]}', name='PizzaOrder'), type='function')]))], created=1731553446, model='gemini-1.5-flash', object='chat.completion', service_tier=None, system_fingerprint=None, usage=None)


In [57]:
# Json format response

# Extract the tool call data from the response
tool_call = response.choices[0].message.tool_calls[0]

# Get the function call arguments and parse as JSON
function_arguments = tool_call.function.arguments
parsed_arguments = json.loads(function_arguments)

# Print the resulting JSON
print(json.dumps(parsed_arguments, indent=4))

{
    "type": "dessert",
    "size": "large",
    "ingredients": [
        "apple",
        "chocolate"
    ]
}


## 6.3 Chain Of Thought (CoT)

In [71]:
model = 'gemini-1.5-flash-latest'

user_prompt_no_CoT = """When I was 4 years old, my partner was 3 times my age. Now, I
am 20 years old. How old is my partner? Return the answer immediately."""

conversation_history = []

response, _ = chat_with_openai_with_retry(user_prompt_no_CoT, 
                                          conversation_history, 
                                          model)

print("No CoT: ", response)
print("--"*20)

user_prompt_with_CoT = """When I was 4 years old, my partner was 3 times my age. Now,
I am 20 years old. How old is my partner? Let's think step by step."""
conversation_history = []
response, _ = chat_with_openai_with_retry(user_prompt_with_CoT, 
                                       conversation_history, 
                                       model)
print("With CoT ", response)

No CoT:  52 

----------------------------------------
With CoT  Here's how to figure this out:

* **Partner's age when you were 4:** Your partner was 3 times your age, so they were 4 * 3 = 12 years old.
* **Age difference:** This means your partner is 12 - 4 = 8 years older than you.
* **Partner's current age:**  Since you are now 20, your partner is 20 + 8 = **28 years old**. 



## 6.4 ReAct: Reason and Act



In [73]:
model_instructions = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation,
Observation is understanding relevant information from an Action's output and Action can be one of three types:
 (1) <search>entity</search>, which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it
     will return some similar entities to search and you can try to search the information from those topics.
 (2) <lookup>keyword</lookup>, which returns the next sentence containing keyword in the current context. This only does exact matches,
     so keep your searches short.
 (3) <finish>answer</finish>, which returns the answer and finishes the task.
"""

example1 = """Question
Musician and satirist Allie Goertz wrote a song about the "The Simpsons" character Milhouse, who Matt Groening named after who?

Thought 1
The question simplifies to "The Simpsons" character Milhouse is named after who. I only need to search Milhouse and find who it is named after.

Action 1
<search>Milhouse</search>

Observation 1
Milhouse Mussolini Van Houten is a recurring character in the Fox animated television series The Simpsons voiced by Pamela Hayden and created by Matt Groening.

Thought 2
The paragraph does not tell who Milhouse is named after, maybe I can look up "named after".

Action 2
<lookup>named after</lookup>

Observation 2
Milhouse was named after U.S. president Richard Nixon, whose middle name was Milhous.

Thought 3
Milhouse was named after U.S. president Richard Nixon, so the answer is Richard Nixon.

Action 3
<finish>Richard Nixon</finish>
"""

example2 = """Question
What is the elevation range for the area that the eastern sector of the Colorado orogeny extends into?

Thought 1
I need to search Colorado orogeny, find the area that the eastern sector of the Colorado orogeny extends into, then find the elevation range of the area.

Action 1
<search>Colorado orogeny</search>

Observation 1
The Colorado orogeny was an episode of mountain building (an orogeny) in Colorado and surrounding areas.

Thought 2
It does not mention the eastern sector. So I need to look up eastern sector.

Action 2
<lookup>eastern sector</lookup>

Observation 2
The eastern sector extends into the High Plains and is called the Central Plains orogeny.

Thought 3
The eastern sector of Colorado orogeny extends into the High Plains. So I need to search High Plains and find its elevation range.

Action 3
<search>High Plains</search>

Observation 3
High Plains refers to one of two distinct land regions

Thought 4
I need to instead search High Plains (United States).

Action 4
<search>High Plains (United States)</search>

Observation 4
The High Plains are a subregion of the Great Plains. From east to west, the High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130m).

Thought 5
High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer is 1,800 to 7,000 ft.

Action 5
<finish>1,800 to 7,000 ft</finish>
"""

In [83]:
model = 'gemini-1.5-flash-latest'

question = """Question
Who was the youngest author listed on the transformers NLP paper?
"""

conversation_history = [{"role": "user", "content": dedent(model_instructions)},
                        {"role": "user", "content": dedent(example1)},
                        {"role": "user", "content": dedent(example2)},
                       ]

# stop: Takes a list of strings where the model should stop generating text.
params= dict(stop=["\nObservation"])
response, conversation_history = chat_with_openai_with_retry(dedent(question), 
                                          conversation_history, 
                                          model,
                                         **params)

print(response)

Thought 1: I need to find the Transformers paper and identify the authors, then figure out who is the youngest.

Action 1: <search>Transformers paper</search>



In [84]:
# Now you can perform this research yourself and supply it back to the model.

observation = """Observation 1
[1706.03762] Attention Is All You Need
Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
We propose a new simple network architecture, the Transformer, based solely on attention mechanisms, dispensing with recurrence and convolutions entirely.
"""

response, conversation_history = chat_with_openai_with_retry(dedent(observation), 
                                          conversation_history, 
                                          model,
                                         **params)

print(response)

Thought 2: I need to look for the authors and try to determine the youngest. Unfortunately, the paper itself does not give information on ages.

Action 2: <search>Ashish Vaswani</search> 



# 7 Code prompting

## 7.1 Generating code

In [88]:
conversation_history = [{"role": "system", "content": "Act as a software developer. Stick to coding only!"}]  # modified

# Gemini 1.5 models are very chatty, so it helps to specify they stick to the code.
code_prompt = """
Write a Python function to calculate the factorial of a number.
"""

params = dict(temperature=1, top_p=1, max_tokens=1024)

response, conversation_history = chat_with_openai_with_retry(code_prompt, 
                                                             conversation_history, 
                                                             model,
                                                             **params)

Markdown(response)

```python
def factorial(n):
  """Calculates the factorial of a non-negative integer.

  Args:
    n: The non-negative integer for which to calculate the factorial.

  Returns:
    The factorial of n, or 1 if n is 0.
  """
  if n < 0:
    raise ValueError("Factorial is not defined for negative numbers.")
  elif n == 0:
    return 1
  else:
    return n * factorial(n - 1)
```

## 7.2 Generate and run code

openAI API does not have a built-in `code_execution` tool, so we'll write the required functionalities

In [137]:
import subprocess


class ExecuteCodeInput(BaseModel):
    """Define the Pydantic model for code execution"""
    code: str = Field(description="The Python code to execute.")


def execute_code_locally(code: str) -> dict:
    """Define the function to execute Python code"""
    try:
        result = subprocess.run(
            ["python3", "-c", code],
            capture_output=True,
            text=True,
            check=True
        )
        return result.stdout.strip()
    except subprocess.CalledProcessError as e:
        print(e)
        return None



code_exec_prompt = """
Calculate the sum of the first 14 prime numbers. Only consider the odd primes, and make sure you count them all.
"""

execute_code_tool = convert_pydantic_to_openai_function(ExecuteCodeInput)
response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": "You are an assistant that writes Python code to solve mathematical problems."},
        {"role": "user", "content": code_exec_prompt}],
        tools=[{"type": "function", "function": execute_code_tool}]
)

# Step 1: Extract the tool call arguments
tool_call = response.choices[0].message.tool_calls[0]
arguments = tool_call.function.arguments

# Step 2: Parse the JSON to get the Python code
code_snippet = json.loads(arguments)['code']
result = execute_code_locally(code_snippet)

print("Result ", result)

## 7.3 Explaining code¶


In [141]:
file_contents = !curl https://raw.githubusercontent.com/magicmonty/bash-git-prompt/refs/heads/master/gitprompt.sh

conversation_history = [{"role": "system", "content": "Act as a software developer."}]  # modified
explain_prompt = f"""
Please explain what this file does at a very high level. What is it, and why would I use it?

```
{file_contents}
```
"""

response, conversation_history = chat_with_openai_with_retry(explain_prompt, 
                                                             conversation_history, 
                                                             model)

Markdown(response)

This is a Bash script designed to integrate Git repository information into your shell prompt. It aims to make your shell more interactive and informative by displaying useful Git details directly in your prompt.

Here's a high-level explanation:

**What it does:**

* **Sets up a color scheme:**  The script defines color themes to customize the appearance of the Git information in your prompt. You can select from built-in themes or create your own custom theme.
* **Retrieves Git status:**  It uses `git` commands to gather information about the current Git repository, including:
    * **Branch name:**  Identifies the branch you're working on.
    * **Upstream status:**  Indicates if your branch is ahead or behind the remote branch.
    * **Staged and unstaged changes:**  Shows the number of staged or unstaged changes.
    * **Other details:**  It can also include information about stashes, conflicts, and whether the working directory is clean.
* **Customizes the prompt:**  The script uses the gathered Git information to dynamically modify your prompt, showing different elements depending on your settings and the current status of the repository.
* **Virtual environment integration:**  It can display the name of any active virtual environment (e.g., from `venv`, `nvm`, `conda`) within your prompt.
* **Window title integration:**  Optionally, the script can set the window title of your terminal to reflect the current Git repository information.
* **Installs prompt commands:**  The script adds functions to your shell that will automatically update your prompt when you change directories or execute Git commands.

**Why you would use it:**

* **Contextual awareness:**  You can quickly see the current Git branch, upstream status, and changes without needing to run separate `git` commands. This keeps you in context and improves your workflow.
* **Increased efficiency:** The visual cues in your prompt help you keep track of the repository's state, which can prevent errors or make it easier to navigate your project.
* **Customizability:** You can choose a color scheme that suits your preferences, specify what Git information you want to see, and integrate the prompt with your existing shell settings.

Overall, this script is a powerful tool for developers and Git users who want to make their shell prompt more interactive and informative, helping them work more efficiently and stay organized within their Git projects.
